In [1]:
!pip install cleverhans==3.1.0

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent

In [3]:
# Load FashionMNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform),
    batch_size=1000, shuffle=False
)

In [4]:
# Define a ResNet50 model adapted for grayscale images and 10 classes
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.model = models.resnet50(weights=None)  # Use weights=None instead of pretrained=False
        # Modify the first conv layer to accept 1-channel input
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # Remove maxpool layer to reduce spatial downsampling
        self.model.maxpool = nn.Identity()
        # Modify the fully connected layer to output 10 classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
# Adversarial training with CleverHans for PGD attack
def train(model, device, train_loader, optimizer, epsilon, alpha, num_iters, epoch):
    model.train()
    total_loss, total_adv_loss = 0, 0
    correct, correct_adv = 0, 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Standard forward pass
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        total_loss += loss.item()

        # Compute accuracy for clean data
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        # Generate adversarial examples using CleverHans PGD
        adv_data = projected_gradient_descent(model, data, eps=epsilon, eps_iter=alpha, nb_iter=num_iters, norm=np.inf)
        adv_output = model(adv_data)
        adv_loss = F.cross_entropy(adv_output, target)
        total_adv_loss += adv_loss.item()

        # Compute accuracy for adversarial data
        adv_pred = adv_output.argmax(dim=1, keepdim=True)
        correct_adv += adv_pred.eq(target.view_as(adv_pred)).sum().item()

        # Combined loss
        total_batch_loss = (loss + adv_loss) / 2

        # Backward pass and optimization
        total_batch_loss.backward()
        optimizer.step()

        total += target.size(0)

        # Log information for each batch
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] '
                  f'Loss: {loss.item():.4f} | Adv Loss: {adv_loss.item():.4f} | '
                  f'Acc: {100. * correct / total:.2f}% | Adv Acc: {100. * correct_adv / total:.2f}%')

    # Log average loss and accuracy for epoch
    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    avg_acc = 100. * correct / total
    avg_adv_acc = 100. * correct_adv / total
    print(f'==> Epoch: {epoch+1} | Avg Loss: {avg_loss:.4f} | Avg Adv Loss: {avg_adv_loss:.4f} | '
          f'Avg Acc: {avg_acc:.2f}% | Avg Adv Acc: {avg_adv_acc:.2f}%')

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")

Using device: cuda


In [6]:
# Training loop
epochs = 5
epsilon = 0.1  # Maximum perturbation
alpha = 0.01   # Step size for each PGD iteration
num_iters = 40  # Number of PGD iterations
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epsilon, alpha, num_iters, epoch)
    print(f"Epoch {epoch+1}/{epochs} completed\n")

print("Adversarial training completed.")

Epoch: 1 [0/60000] Loss: 2.5078 | Adv Loss: 2.5573 | Acc: 4.69% | Adv Acc: 6.25%
Epoch: 1 [6400/60000] Loss: 5.8380 | Adv Loss: 5.3701 | Acc: 9.95% | Adv Acc: 9.98%
Epoch: 1 [12800/60000] Loss: 4.6358 | Adv Loss: 4.4435 | Acc: 9.94% | Adv Acc: 9.96%
Epoch: 1 [19200/60000] Loss: 4.0319 | Adv Loss: 3.8973 | Acc: 10.00% | Adv Acc: 10.01%
Epoch: 1 [25600/60000] Loss: 3.6543 | Adv Loss: 3.5647 | Acc: 9.98% | Adv Acc: 9.98%
Epoch: 1 [32000/60000] Loss: 3.8223 | Adv Loss: 3.7555 | Acc: 10.05% | Adv Acc: 10.06%
Epoch: 1 [38400/60000] Loss: 4.1039 | Adv Loss: 4.0061 | Acc: 10.07% | Adv Acc: 10.07%
Epoch: 1 [44800/60000] Loss: 3.9773 | Adv Loss: 3.9193 | Acc: 10.10% | Adv Acc: 10.11%
Epoch: 1 [51200/60000] Loss: 3.7504 | Adv Loss: 3.6753 | Acc: 10.01% | Adv Acc: 10.01%
Epoch: 1 [57600/60000] Loss: 3.8910 | Adv Loss: 3.7905 | Acc: 10.00% | Adv Acc: 10.00%
==> Epoch: 1 | Avg Loss: 4.3657 | Avg Adv Loss: 4.1000 | Avg Acc: 10.00% | Avg Adv Acc: 10.01%
Epoch 1/5 completed

Epoch: 2 [0/60000] Loss: 4.